### 数据预处理：预处理、标准化、纠偏
#### 1、预处理
##### 缺失值：样本的部分特征信息缺失
处理方式：
    - 删除：缺失样本量非常大，删除整个变量，如果缺失量较少，且难以填充则删除缺失样本
    - 填充：缺失量小于10%，根据缺失变量的数据分布采取均值或中位数进行填充
    - 预测缺失样：使用与缺失相比相关性非常高的特征，简历模型，预测缺失值

In [ ]:
#数据预处理演示
import pandas as pd
import matplotlib.pyplot as plt
df = pd.read_csv('')
df.columns
print(df.shape)
print(df.dtype)
print(df.head())


In [ ]:
#缺失值处理
#查看数据样本中的缺失情况
print(df.isnull().sum())

In [ ]:
#逐一查看特征，进行缺失值填充
#查看房价
df['房价'].hist()
plt.show()

In [ ]:
#选择均值填充
price_mean = df['房价'].mean()
print(price_mean)
#使用均值进行缺失值填充
df.loc[df['房价'].isna(),'房价'] =price_mean
print(df.isnull().sum())

In [ ]:
#选择均值填充
area_median = df['area'].median()
print(area_median)
#使用均值进行缺失值填充
df.loc[df['area'].isna(),'area'] =area_median
print(df.isnull().sum())

In [ ]:
#查看相关性系数
print(df[['日间人口','夜间人口','字段3']].corr())

In [ ]:
#训练线性回归模型对夜间认可进行填补
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score  #评价指标r2_score
train = df.copy().loc[df['夜间人口'].isna(),['日间人口','夜间人口']].reset_index(drop=True)
x = train(['日间人口'])
y = train(['夜间人口'])
model =LinearRegression()
model.fit(x,y)
print(f'R squared is: {r2_score(y,x)}')

In [ ]:
#对缺失值进行填充
df.loc[df['夜间人口'].isna(),['夜间人口']] =model.predict(df.loc[df['日间人口'].isna(),['日间人口']])

In [ ]:
#特殊类型的缺失值
print(df.complete_year.value_counts())

In [ ]:
#做切片删除缺失房龄的样本
df = df[df.complete_year != '未知'].reset_index(drop=True)


### 离群值：远离数据主要部分的样本（极大值或极小值）
处理方式：
    - 删除：直接删除离群样本
    - 填充样本：使用box-plot定义变量的数值上下界，以上界填充极大值，下界填充最小值

In [ ]:
#查看房价的离群情况
df['房价'].hist()
plt.show()
df['房价'].boxplot()
plt.show()

In [ ]:
#根据箱线图的上下限进行异常值的填充
def boxplot_fill(col):
    #计算iqr：数据四分之三分位值和四分之一分位值得差
    iqr = col.quantile(0.75) - col.quantile(0.25)
    #根据iqr计算异常值判断阈值
    u_th = col.quantile(0.75) + 1.5*iqr #上界
    l_th = col.quantile(0.25) - 1.5*iqr #下界
    #定义转换函数：如果数字大于上界则用上界值填充，小于下界则用下界值填充
    def box_trans(x):
        if x > u_th:
            return u_th
        elif x <l_th:
            return l_th
        else:
            return x
    return col.map(box_trans)
#填充效果查看
boxplot_fill(df['房价']).hist()
#进行赋值
df['房价'] = boxplot_fill(df['房价'])
plt.show()

In [ ]:
df.to_excel('名称.xlsx',index=False)


### 2、标准化
#### 标准化目的：
    - 去除数据量纲的影响
    - 提高模型的解释性
    - 加快模型收敛速度
#### 标准化的方法
    - 中心化：减去均值再除以标准差
    - 01标准化：减去最小值再除以最大值与最小值得差


In [ ]:
#sklearn 中标准化的方法
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
#以中心化为例子讲解sklearn中标准化的使用方法
scaler = StandardScaler()
df['日间人口'].hist()
plt.show()
trans_data = df.copy()[['日间人口']]
scaler.fit(trans_data)
trans_data['日间人口'] = scaler.transform(trans_data)
trans_data['日间人口'].hist()
plt.show()


In [ ]:
#使用pipeline整合数据标准化与模型
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
#构建模型工作流
pipe_lm = Pipeline([
        ('sc',StandardScaler()),
        ('lm_regr',LinearRegression())
        ])
print(pipe_lm)


#### 3、纠偏
    - 正态分布：数据呈现对称的钟型分布
    - 右偏态 ：样本大量集中在均值左边（均值偏到了右边）
    - 左偏态 ：样本大量集中在均值右边（均值偏到了左边）










